**Model Run** 

In [1]:
# import necessary libraries and model
from model import AdaptationModel
import matplotlib.pyplot as plt
import networkx as nx
from collections import defaultdict 
import pandas as pd

In [48]:
# set-up model parameters
# run time: 100 years --> 400 quarters, so 400 ticks in total
run_length = 400
# set the number of replications 
replication_number = 10
# set the number of households
number_of_households = 500

In [29]:
# use defaultdict to store the results with different seeds
model_results =  defaultdict(list)
agent_results = defaultdict(list)
# Run the model based on the parameters above, no subsidy included
for i in range(replication_number):
    seed = i  # set the seed
    model = AdaptationModel(number_of_households= number_of_households,subsidy_rate=0, income_threshold=2000, flood_map_choice="harvey", network="no_network", seed=seed)
    for tick in range(run_length):
        model.step() 
    model_data = model.datacollector.get_model_vars_dataframe()
    agent_data = model.datacollector.get_agent_vars_dataframe()
    model_results[i].append(model_data)
    agent_results[i].append(agent_data)
        

**Save the Results**

In [31]:
# function to convert the results to dataframe, with one index
def convert_to_dataframe(results, replication_number):
    dfs = [pd.concat(results[i], keys=[i], names=['replication/seed']) for i in range(replication_number)]
    dataframe = pd.concat(dfs)
    dataframe.reset_index(inplace=True)
    dataframe.rename(columns={'level_1': 'Step'}, inplace=True)
    return dataframe

In [32]:
# convert the model results to dataframe
model_dataframe = convert_to_dataframe(model_results, replication_number)
# convert the agent results to dataframe
agent_dataframe = convert_to_dataframe(agent_results, replication_number)

C:\Users\canan\AppData\Local\Temp\ipykernel_26396\3136032148.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat(dfs)


In [87]:
# Save the results to csv files
model_dataframe.to_csv(f"../results/{'model_results_no_subsidy'}.csv", index=False)
agent_dataframe.to_csv(f"../results/{'agent_results_no_subsidy'}.csv", index=False)

In [62]:
# Load the data with the following code
# change the file name to the file you want to load
#model_dataframe = pd.read_csv(f"../results/{'model_results'}.csv")

**Sensitivity Analysis**

In [3]:
# Parameters for the sensitivity analysis
subsidy_rate = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 
                0.7, 0.8, 0.9, 1]   # subsidy rate offered
income_threshold = [0, 1000, 3000, 5000, 8000, 10000] # agents which have lower income then the threshold eligible for subsidy
saving_threshold = [0, 0.1, 0.25, 0.5, 0.75] # agent calculate_saving(self) threshold, for saving or consuming

**1. Saving Threshold Sensitivity**

In [4]:
# use defaultdict to store the results with different seeds
model_results_saving_threshold =  defaultdict(list)
agent_results_saving_threshold = defaultdict(list)
# Run the sensitivity for different saving thresholds, no subsidy included
for rate in saving_threshold:
    for i in range(replication_number):
        seed = i  # set the seed
        model = AdaptationModel(number_of_households= number_of_households,subsidy_rate=0, income_threshold=2000, flood_map_choice="harvey", network="no_network", seed=seed)
        for tick in range(run_length):
            model.step() 
        model_data2 = model.datacollector.get_model_vars_dataframe()
        agent_data2 = model.datacollector.get_agent_vars_dataframe()
        agent_data2.reset_index(inplace=True)
        model_results_saving_threshold[i,rate].append(model_data2)
        agent_results_saving_threshold[i,rate].append(agent_data2)

**Save the Results**

In [19]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs = {}

# Concatenate the dataframes for each (i, rate) key
for key, result_list in model_results_saving_threshold.items():
    i, rate = key
    concatenated_dfs[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
model_dataframe_saving_threshold = pd.concat(concatenated_dfs.values(), keys=concatenated_dfs.keys(), names=['replication/seed', 'rate', 'Step'])

# Reset the index 
model_dataframe_saving_threshold.reset_index(inplace=True)


In [16]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs2 = {}

# Concatenate the dataframes for each (i, rate) key
for key, result_list in agent_results_saving_threshold.items():
    i, rate = key
    concatenated_dfs2[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
agent_dataframe_saving_threshold = pd.concat(concatenated_dfs2.values(), keys=concatenated_dfs2.keys(), names=['replication/seed', 'rate', 'index'])

# Reset the index 
agent_dataframe_saving_threshold.reset_index(inplace=True)
# drop index column
agent_dataframe_saving_threshold.drop(columns=['index'], inplace=True)


C:\Users\canan\AppData\Local\Temp\ipykernel_26396\1444048260.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  agent_dataframe_saving_threshold = pd.concat(concatenated_dfs2.values(), keys=concatenated_dfs2.keys(), names=['replication', 'rate', 'index'])


In [97]:
# Save the results to csv files
model_dataframe_saving_threshold.to_csv(f"../results/{'model_sensitivity_results_saving_threshold'}.csv", index=False)
agent_dataframe_saving_threshold.to_csv(f"../results/{'agent_sensitivity_results_saving_threshold'}.csv", index=False)

In [100]:
#upload the data with the following code
# change the file name to the file you want to load
#model_dataframe_saving_threshold = pd.read_csv(f"../results/{'model_sensitivity_results_saving_threshold'}.csv")

**2.Income Threshold**

In [18]:
# use defaultdict to store the results with different seeds
model_results_income_threshold =  defaultdict(list)
agent_results_income_threshold = defaultdict(list)
# Run the sensitivity for different income thresholds, when subsidy is 10%
for income in income_threshold:
    for i in range(replication_number):
        seed = i  # set the seed
        model = AdaptationModel(number_of_households= number_of_households,subsidy_rate=0.1, income_threshold=income, flood_map_choice="harvey", network="no_network", seed=seed)
        for tick in range(run_length):
            model.step() 
        model_data3 = model.datacollector.get_model_vars_dataframe()
        agent_data3 = model.datacollector.get_agent_vars_dataframe()
        agent_data3.reset_index(inplace=True)
        model_data3
        model_results_income_threshold[i,income].append(model_data3)
        agent_results_income_threshold[i,income].append(agent_data3)

In [21]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs = {}

# Concatenate the dataframes for each (i, rate) key
for key, result_list in model_results_income_threshold.items():
    i, rate = key
    concatenated_dfs[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
model_dataframe_income_threshold = pd.concat(concatenated_dfs.values(), keys=concatenated_dfs.keys(), names=['replication/seed', 'income_threshold', 'Step'])

# Reset the index 
model_dataframe_income_threshold.reset_index(inplace=True)



In [22]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs2 = {}

# Concatenate the dataframes for each (i, rate) key
for key, result_list in agent_results_income_threshold.items():
    i, rate = key
    concatenated_dfs2[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
agent_dataframe_income_threshold = pd.concat(concatenated_dfs2.values(), keys=concatenated_dfs2.keys(), names=['replication/seed', 'income_threshold', 'index'])

# Reset the index 
agent_dataframe_income_threshold.reset_index(inplace=True)
# drop index column
agent_dataframe_income_threshold.drop(columns=['index'], inplace=True)


C:\Users\canan\AppData\Local\Temp\ipykernel_26396\1200854002.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  agent_dataframe_income_threshold = pd.concat(concatenated_dfs2.values(), keys=concatenated_dfs2.keys(), names=['replication/seed', 'income_threshold', 'index'])


In [35]:
# save the results to csv files
model_dataframe_income_threshold.to_csv(f"../results/{'model_sensitivity_results_income_threshold'}.csv", index=False)
agent_dataframe_income_threshold.to_csv(f"../results/{'agent_sensitivity_results_income_threshold'}.csv", index=False)

In [38]:
# upload the data with the following code
# change the file name to the file you want to load
# model_dataframe_income_threshold = pd.read_csv(f"../results/{'model_sensitivity_results_income_threshold'}.csv")

**3.Subsidy Rate**

In [39]:
# use defaultdict to store the results with different seeds
model_results_subsidy_rate =  defaultdict(list)
agent_results_subsidy_rate = defaultdict(list)
# Run the sensitivity for different income thresholds, when income is 3000
for subsidy in subsidy_rate:
    for i in range(replication_number):
        seed = i  # set the seed
        model = AdaptationModel(number_of_households= number_of_households,subsidy_rate=subsidy, income_threshold=3000, flood_map_choice="harvey", network="no_network", seed=seed)
        for tick in range(run_length):
            model.step() 
        model_data4 = model.datacollector.get_model_vars_dataframe()
        agent_data4 = model.datacollector.get_agent_vars_dataframe()
        agent_data4.reset_index(inplace=True)
        model_data4
        model_results_subsidy_rate[i,subsidy].append(model_data4)
        agent_results_subsidy_rate[i,subsidy].append(agent_data4)

In [40]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs = {}

# Concatenate the dataframes for each (i, rate) key
for key, result_list in model_results_subsidy_rate.items():
    i, rate = key
    concatenated_dfs[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
model_dataframe_subsidy_rate = pd.concat(concatenated_dfs.values(), keys=concatenated_dfs.keys(), names=['replication/seed', 'subsidy_rate', 'Step'])

# Reset the index 
model_dataframe_subsidy_rate.reset_index(inplace=True)



In [42]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs2 = {}

# Concatenate the dataframes for each (i, rate) key
for key, result_list in agent_results_subsidy_rate.items():
    i, rate = key
    concatenated_dfs2[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
agent_dataframe_subsidy_rate = pd.concat(concatenated_dfs2.values(), keys=concatenated_dfs2.keys(), names=['replication/seed', 'subsidy_rate', 'index'])

# Reset the index 
agent_dataframe_subsidy_rate.reset_index(inplace=True)
# drop index column
agent_dataframe_subsidy_rate.drop(columns=['index'], inplace=True)


C:\Users\canan\AppData\Local\Temp\ipykernel_26396\1714474735.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  agent_dataframe_subsidy_rate = pd.concat(concatenated_dfs2.values(), keys=concatenated_dfs2.keys(), names=['replication/seed', 'subsidy_rate', 'index'])


In [44]:
# save the results to csv files
model_dataframe_subsidy_rate.to_csv(f"../results/{'model_sensitivity_results_subsidy_rate'}.csv", index=False)
agent_dataframe_subsidy_rate.to_csv(f"../results/{'agent_sensitivity_results_subsidy_rate'}.csv", index=False)

In [47]:
# upload the data with the following code
# change the file name to the file you want to load
# model_dataframe_subsidy_rate = pd.read_csv(f"../results/{'model_sensitivity_results_subsidy_rate'}.csv")